Importação das bibliotecas necessárias.
requests: para "chamar" a URL
bs4: possibilita buscar tags específicas na URL que o rquests "chamou"
pandas: usado para manipular DataFrames
unicodedata: usado para configurar os textos (retirando acentos)

In [ ]:
"""
web scraping do site vlr.gg para capturar os ultimos jogos dos 128 principais times de valorant
"""
import requests
from bs4 import BeautifulSoup
import pandas as pd
import unicodedata

In [ ]:
Função que captura as informações gerais do site

In [2]:
def capturar_dados(url):
    """funcao responsavel por capturar as informacoes gerais do site a partir da url especifica"""
    resposta = requests.get(url)

    # Verificar se a requisicao foi bem-sucedida
    if resposta.status_code == 200:
        # Analisa o conteudo HTML da página
        soup = BeautifulSoup(resposta.text, 'html.parser')

        # busca todos os jogos registrados na URL
        jogos_total = soup.select_one('div.mod-dark')

        return jogos_total

    # retorna uma lista vazia caso nao encontre partidas
    return []

In [ ]:
A partir de um loop for, cada time e partidas desse time são buscados no html 

In [3]:
def capturar_pagina():
    """funcao que captura as urls das paginas especificas de cada time"""
    # configura o nome da equipe
    total = []

    site = requests.get('https://www.vlr.gg/rankings')

    if site.status_code == 200:
        captura = BeautifulSoup(site.text, 'html.parser')

        time = captura.find_all('div', class_='world-rankings-col')

        for linha in time:
            # busca no site se o time esta listado
            busca = linha.find_all('tr')

            # passa por todas as tags 'tr' para identificar os times e suas urls
            for g in busca:
                try:
                    time = g.find('td', class_='rank-item-team')
                    name = time.get('data-sort-value')

                    # captura a parte da url que ira direcionar para a pagina de partidas de cada time
                    referencia = time.find('a')
                    href = referencia.get('href')

                    # lista que contem o nome do time e a url da sua pagina de partidas
                    total.append([name, href])
                except AttributeError:
                    # continua a execucao caso algum bloco 'tr' do site não possuir informacoes
                    pass
    else:
        print('Request falhou.')

    return total

In [ ]:
Momento de uso da biblioteca unicodedata para corrigir textos para um formatouniforme para melhor manipulação dos dados

In [4]:
def normaliza_nomes(nome: str) -> str:
    """funcao para organizar o nome do time para compor o nome do arquivo"""
    n = nome.replace(' ', '_').lower()
    # Normaliza a string para a forma NFD (Normalization Form Decomposition)
    texto = unicodedata.normalize('NFD', n)
    # Remove caracteres diacríticos
    texto_sem_acento = ''.join([c for c in texto if not unicodedata.combining(c)])

    return texto_sem_acento

In [ ]:
Esta célula é responsável por capturar a URL, adicionar a URL do time o caminho para as pastidas além de gerar mais 5 (cindo) 
URLs (considerando as páginas que possuem os históricos de partidas, limitadas a cinco páginas de pesquisa) do time.

In [9]:
# captura o nome dos principais times e a url que se encontra as informacoes a respeito dele
fonte = capturar_pagina()

# adiciona a url do time o campo 'matches' que é o local onde se encontram todas as partidas do time
for item in fonte:
    item[1] = item[1][:6] + 'matches/' + item[1][6:]

# adiciona paginas a url para capturar jogos de outras paginas, captura ate a 5 pagina
for f in range(len(fonte)):
    pop = fonte[f].pop(-1)
    for k in range(1, 6):
        fonte[f].append('https://www.vlr.gg' + pop + '/?page=' + str(k))

In [ ]:
Principal parte do código, passa por cada time listado e com esse time em foco, capturas o histórico e salva em um arquivo .csv,
imprime o nome do time e a quantidade de jogos por página encontrada.

In [14]:
# Realizar uma requisição GET para a URL
for t in fonte:
    print(f'\nTime: {t[0]}')
    validacao_time_casa = 0

    # lista que armazenará todas as partidas encontradas na URL
    total_partidas = []

    for i in t[1:]:
        # função que captura os dados da url
        try:
            corpo_jogos = capturar_dados(i)
            jogos = corpo_jogos.find_all('a', class_='wf-card fc-flex m-item')

            # Imprime o total de partidas da URL, caso tenha
            if len(jogos) > 0:
                print('Quantidade de partidas: ', len(jogos))

            else:
                print('Erro ao capturar partidas')

            # Iteração para encontrar, em cada jogo, o time oponente e o placar da partida
            for jogo in jogos:

                # Time contra
                info_jogos = jogo.find('div', class_='m-item-team text-of mod-right')
                time_oponente = info_jogos.find('span', class_='m-item-team-name').text.strip()

                # captura a data que ocorreu o jogo
                data_classe = jogo.find('div', class_='m-item-date')
                data = data_classe.find('div').text.strip()

                # Se perdeu na serie
                placar_geral = jogo.find('div', class_='m-item-result mod-loss fc-flex mod-expandable mod-collapsed')

                # Se ganhou na serie
                if not placar_geral:
                    placar_geral = jogo.find('div', class_='m-item-result mod-win fc-flex mod-expandable mod-collapsed')

                # O placar está em 2 'spans'
                spans = placar_geral.find_all('span')

                span0 = spans[0].text.strip()
                span1 = spans[1].text.strip()

                # Uma tupla contendo placar (time casa e time visitante) e o nome do time visitante
                # spans[0] é o placar oponente, spans[1] é o placar da Furia
                total_partidas.append((t[0], span0, span1, time_oponente, data))

        except:
            # continua a execucao caso algum bloco do site não possuir informacoes
            pass

    # converte os dados capturados em um dataFrame
    df = pd.DataFrame(total_partidas, columns=['equipe', 'placar_casa', 'placar_visita', 'time_visita', 'data'])
    df.to_csv(f'historicos\\historico_{normaliza_nomes(t[0])}.csv', sep=';')


Time: Team Heretics
Quantidade de partidas:  50
Quantidade de partidas:  50
Quantidade de partidas:  50
Quantidade de partidas:  19

Time: FNATIC
Quantidade de partidas:  50
Quantidade de partidas:  50
Quantidade de partidas:  49

Time: DSYRE
Quantidade de partidas:  50
Quantidade de partidas:  50
Quantidade de partidas:  50
Quantidade de partidas:  33

Time: FUT Esports
Quantidade de partidas:  50
Quantidade de partidas:  50
Quantidade de partidas:  41

Time: CGN Esports
Quantidade de partidas:  50
Quantidade de partidas:  50
Quantidade de partidas:  50
Quantidade de partidas:  50
Quantidade de partidas:  33

Time: GoNext Esports
Quantidade de partidas:  50
Quantidade de partidas:  50
Quantidade de partidas:  36

Time: MOUZ
Quantidade de partidas:  50
Quantidade de partidas:  44

Time: Fire Flux Esports
Quantidade de partidas:  50
Quantidade de partidas:  50
Quantidade de partidas:  50
Quantidade de partidas:  22

Time: Natus Vincere
Quantidade de partidas:  50
Quantidade de partidas